<a href="https://colab.research.google.com/github/shu0518/Gen_AI_final_m11423036/blob/main/%E7%94%9F%E6%88%90%E5%BC%8F_AI_%E6%9C%9F%E6%9C%AB%E5%B0%88%E6%A1%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AI 教授分身：基於 RAG 的課程智慧助教

### 1. 讀入 vector dataset

In [ ]:
import gdown
import re
import os

GDRIVE_FILE_URL = "https://drive.google.com/file/d/1a5tMEytl677vrJPgBk8d9l-aMwL8jJ6R/view?usp=sharing"

# 自動提取 ID
def get_id_from_url(url):
    match = re.search(r'/d/([a-zA-Z0-9-_]+)', url)
    return match.group(1) if match else url

file_id = get_id_from_url(GDRIVE_FILE_URL)

print(f"🚀 開始下載資料庫 (ID: {file_id})...")
# 使用 gdown 下載並命名為 faiss_db.zip
gdown.download(id=file_id, output="faiss_db.zip", quiet=False)

# 解壓縮
if os.path.exists("faiss_db.zip"):
    !unzip -o faiss_db.zip
    print("✅ 資料庫解壓縮完成！")
else:
    print("❌ 下載失敗，請檢查連結權限是否設為「公開」。")

🚀 開始下載資料庫 (ID: 1a5tMEytl677vrJPgBk8d9l-aMwL8jJ6R)...


Downloading...
From: https://drive.google.com/uc?id=1a5tMEytl677vrJPgBk8d9l-aMwL8jJ6R
To: /content/faiss_db.zip
100%|██████████| 7.27M/7.27M [00:00<00:00, 65.8MB/s]


Archive:  faiss_db.zip
  inflating: faiss_db/index.pkl      
  inflating: faiss_db/index.faiss    
✅ 資料庫解壓縮完成！


### 2. 安裝並引入必要套件

In [ ]:
!pip install -U langchain langchain-community faiss-cpu transformers sentence-transformers huggingface_hub
!pip -q install "aisuite[all]"

import os
import gradio as gr
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.chat_models import ChatOpenAI
from openai import OpenAI
from huggingface_hub import login
from google.colab import userdata
import aisuite as ai

# 設定 API Key
hf_token = userdata.get('HuggingFace')
login(token=hf_token)

print("✅ 套件與環境設定完成")

  Using cached huggingface_hub-1.2.2-py3-none-any.whl.metadata (13 kB)
✅ 套件與環境設定完成


### 3. 加入 EmbeddingGemma 前綴

In [ ]:
class EmbeddingGemmaEmbeddings(HuggingFaceEmbeddings):
    def __init__(self, **kwargs):
        super().__init__(
            model_name="google/embeddinggemma-300m",
            encode_kwargs={"normalize_embeddings": True},
            **kwargs
        )

    def embed_documents(self, texts):
        texts = [f"title: none | text: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        # 官方檢索建議前綴
        return super().embed_query(f"task: search result | query: {text}")

### 4. 載入 `faiss_db`

In [ ]:
try:
    embedding_model = EmbeddingGemmaEmbeddings()
    vectorstore = FAISS.load_local(
        "faiss_db",
        embeddings=embedding_model,
        allow_dangerous_deserialization=True
    )
    # k=4 代表一次抓 4 筆最相關的資料
    retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
    print("✅ 向量資料庫載入成功！")
except Exception as e:
    print(f"❌ 載入失敗: {e}")
    print("💡 請確認 'faiss_db.zip' 是否已成功解壓縮。")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ 向量資料庫載入成功！


### 5. 設定要用的 LLM

In [ ]:
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key

client = ai.Client()

# 設定模型：使用 Groq 上的 Llama 3.3 (中文與推論能力極佳)
model = "groq:llama-3.3-70b-versatile"

print(f"✅ LLM 設定完成，使用模型: {model}")

✅ LLM 設定完成，使用模型: groq:llama-3.3-70b-versatile


### 6.系統 `prompt` 設計

In [ ]:
# System Prompt: 教授風格模仿
system_prompt = """
你現在是「生成式 AI：文字與圖像生成的原理與實務」的授課教授。
你的任務是根據檢索到的教材回答學生問題，但最重要的任務是「還原教授的上課語氣」。

請務必模仿以下這位教授獨特的說話習慣：

【說話習慣設定】(請嚴格遵守)：
1. 核心口頭禪 (必須使用)：
   - 做總結或轉換話題時，請務必說：「總而言之言而總之」。
   - 提到潛在風險或錯誤時，請重複強調：「這個很容易出亂子，很容易出亂子」。
   - 假設情況時，請說：「萬一萬一」 (重複兩次)。

2. 語句結構：
   - 說話要有「碎念感」，句子之間多用「好，那...」、「但是呢...」、「也就是說...」來連接。
   - 偶爾可以稍微重複前面的詞，例如：「我們準備要...我們就準備要...」。

3. 開場與結尾：
   - 開頭可以用：「好，那我們現在...」或是「我們今天要做的事情...」。
   - 語氣要像是在跟學生耳提面命，帶有一點點擔心學生做錯的關心。

【回答功能規則】：
1. 引用來源：回答時，請明確指出資訊來自哪一堂課 (例如：「根據第14講的簡報...」)。
2. 數學公式：遇到數學符號務必用 `$` 包裹 (例如 $E=mc^2$)，以便正確顯示。
3. 誠實原則：若教材沒提到，請直說「這個部分投影片好像沒有特別講」。
"""

# Template: 注入檢索到的資料
prompt_template = """
請根據以下檢索到的課程教材內容，以教授的口吻回答：

【課程教材片段】：
{retrieved_chunks}

【學生問題】：{question}
"""

### 7. 使用 RAG 來回應

搜尋與使用者問題相關的資訊，根據我們的 prompt 樣版去讓 LLM 回應。

In [ ]:
def chat_with_rag(user_input, history):
    # history 參數雖然這裡沒用到，但為了配合 Gradio 的呼叫格式，必須保留

    # 1. 取回相關資料
    docs = retriever.invoke(user_input)

    # 2. 整理檢索到的內容
    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    # 3. 將自定 prompt 套入格式
    final_prompt = prompt_template.format(
        retrieved_chunks=retrieved_chunks,
        question=user_input
    )

    # 4. 用 AI Suite 呼叫語言模型
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": final_prompt},
        ],
        temperature=0.3
    )

    return response.choices[0].message.content

### 8. 用 Gradio 打造 Web App

In [ ]:
IMAGE_URL = "https://drive.google.com/file/d/1CQ0DYnToX0KBVeasxCZOvU9VT0ZpEknu/view?usp=sharing"

# 下載頭像
print("🚀 正在下載AI助教頭像...")
img_id = get_id_from_url(IMAGE_URL)
# 下載並命名為 wizard_prof.png
gdown.download(id=img_id, output="professor_avatar.jpg", quiet=False)
if os.path.exists("professor_avatar.jpg"):
    print("✅ 頭像圖片準備完成")
else:
    print("⚠️ 頭像下載失敗，將使用預設圖示")

🚀 正在下載魔法師頭像...


Downloading...
From: https://drive.google.com/uc?id=1CQ0DYnToX0KBVeasxCZOvU9VT0ZpEknu
To: /content/professor_avatar.jpg
100%|██████████| 350k/350k [00:00<00:00, 71.6MB/s]

✅ 頭像圖片準備完成


In [ ]:
# 🧙‍♂️ 設定魔法風格主題

magic_theme = gr.themes.Soft(
    primary_hue="purple",   # 主色：魔法紫
    secondary_hue="amber",  # 副色：鍊金金
    neutral_hue="slate",    # 底色：深岩灰
    font=[gr.themes.GoogleFont("Noto Sans TC"), "sans-serif"],
).set(
    # 強制設定深色模式的顏色代碼
    body_background_fill="#0F172A",
    body_text_color="#E2E8F0",
    block_background_fill="#1E293B",
    block_border_color="#6B4C9A",
    block_title_text_color="#FCD34D",
    input_background_fill="#334155",
    button_primary_background_fill="#6B4C9A",
    button_primary_text_color="#FFFFFF",
    border_color_primary="#FFC107",
)

# ✨ 自定義 CSS (針對頭像與文字調整)

custom_css = """
/* 歡迎語樣式 */
.welcome-msg {
    color: #CBD5E1 !important;
    font-size: 1.1rem;
    text-align: center;
    margin-top: 10px;
    margin-bottom: 20px;
}

/* 標題樣式 */
h1 {
    color: #FCD34D !important;
    text-align: center;
}
"""

# 🚀 啟動 Gradio 介面

with gr.Blocks(theme=magic_theme, css=custom_css, title="生成式 AI 智慧助教") as demo:

    # --- 標題與歡迎語區 ---
    with gr.Row():
        with gr.Column():
            gr.Markdown(
                """
                <h1>🤖 生成式 AI 智慧助教</h1>
                <p class="welcome-msg">
                    這裡儲存了課程所有的知識結晶，有什麼問題就儘管問！
                </p>
                """
            )

    # --- 主操作區 ---
    with gr.Row():
        # 左側：聊天視窗
        with gr.Column(scale=4):
            chatbot = gr.Chatbot(
                label="AI 助教",
                height=600,
                type="messages",
                # 👇 請確認圖片檔名是否正確
                latex_delimiters=[
                    {"left": "$$", "right": "$$", "display": True},
                    {"left": "$", "right": "$", "display": False},
                    {"left": "\\(", "right": "\\)", "display": False},
                    {"left": "\\[", "right": "\\]", "display": True}
                ]
            )

    # --- 輸入與按鈕區 ---
    with gr.Row():
        msg = gr.Textbox(
            placeholder="請輸入關於課程的問題... (Shift+Enter 可以換行)",
            label="你的提問",
            lines=1,
            max_lines=5,
            scale=4
        )
        submit_btn = gr.Button("送出問題", variant="primary", scale=1)

    # --- 範例區 ---
    with gr.Row():
        gr.Examples(
            examples=[
                "Transformer 的架構是什麼？",
                "最好的生圖模型是什麼？",
                "請解釋 Attention 機制的數學原理",
                "什麼是 RAG？",
            ],
            inputs=msg,
            label="範例問題"
        )

    # --- 邏輯處理 ---
    def respond(message, chat_history):
        if not message.strip():
            return "", chat_history

        try:
            bot_message = chat_with_rag(message, chat_history)
        except Exception as e:
            bot_message = f"❌ 發生錯誤：{str(e)}。請檢查連線。"

        chat_history.append({"role": "user", "content": message})
        chat_history.append({"role": "assistant", "content": bot_message})
        return "", chat_history

    # 綁定事件
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    submit_btn.click(respond, [msg, chatbot], [msg, chatbot])

# 啟動
print("🚀 生成式 AI 智慧助教啟動中...")
demo.launch(debug=True, share=True)

/tmp/ipython-input-1415021315.py:42: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=magic_theme, css=custom_css, title="生成式 AI 智慧助教") as demo:
/tmp/ipython-input-1415021315.py:42: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(theme=magic_theme, css=custom_css, title="生成式 AI 智慧助教") as demo:
/tmp/ipython-input-1415021315.py:60: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(


🚀 生成式 AI 智慧助教啟動中...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2513620e833ab5bc01.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2513620e833ab5bc01.gradio.live


### Gradio 的對話結果